In [2]:
# ! pip install kaggle

In [3]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/N/u/tanakulk/Carbonate/.kaggle’: File exists


In [4]:
! cp kaggle.json ~/.kaggle/
#! cp /kaggle.json ~/.kaggle/

cp: cannot stat ‘kaggle.json’: No such file or directory


In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle competitions download facebook-recruiting-iii-keyword-extraction -f Train.zip

Train.zip: Skipping, found more recently modified local copy (use --force to force download)


In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import ClassifierChain
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn.metrics import jaccard_score
from sklearn.metrics import classification_report
import numpy as np
import time

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

#pip install scikit-multilearn --upgrade

[nltk_data] Downloading package punkt to
[nltk_data]     /N/u/tanakulk/Carbonate/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /N/u/tanakulk/Carbonate/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /N/u/tanakulk/Carbonate/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /N/u/tanakulk/Carbonate/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [8]:
# ! unzip Train.zip

In [2]:
train_data = pd.read_csv("Train.csv")
len(train_data)

6034195

Dropping duplicate values present in the dataset

In [3]:
data = train_data.drop_duplicates(subset = ["Title","Tags"])
len(data)

4186184

Dropping rows with no tags

In [4]:
data['Tags'].isna().sum()

7

In [5]:
data = data.dropna(subset=['Tags'])

Using 200k records for computation purposes

In [32]:
data_split = data.loc[:200000]

In [33]:
data_split = data_split.reset_index(drop = True)

Function to perform Lemmatization using WordNetLemmatizer

In [34]:
def lemmatize(name):
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV

    lemmatizer = WordNetLemmatizer()

    lemmatized = []

    for i in range(len(data_split)):
      ls = ""
      for word, tag in pos_tag(data_split[name][i]):
        ls += " " + lemmatizer.lemmatize(word, tag_map[tag[0]])
      lemmatized.append(ls)
    return lemmatized

Function to complete the following pre-processnig steps:
1. Remove HTML tags using Beautiful Soup
2. Remove special characters using regular expressions
3. Convert text to its lower form
4. Tokenize the text
5. Removing stopwords except 'C' and 'R'
6. Performing Lemmatization

In [35]:
def preprocessing(columnname, name):
    
    cleantext = []
    for i in range(len(data_split)):
      cleantext.append(BeautifulSoup(data_split[columnname][i], "html").text)
    
    data_split[name] = cleantext
    
    data_split[name] = data_split[name].apply(lambda x : re.sub(r"[^a-zA-Z0-9]+", ' ', x))
    
    data_split[name] = data_split[name].apply(lambda s: s.lower() if type(s) == str else s)
    
    data_split[name] = data_split[name].apply(lambda x : word_tokenize(x))
    
    stopword = stopwords.words('english')
    data_split[name] = data_split[name].apply(lambda x : [word for word in x if not word in stopword or word=='c' or word=='r'])
    
    name_new = name + " " + "tokens"
    data_split[name_new] = lemmatize(name)


In [36]:
preprocessing('Body','clean body text')
preprocessing('Title','clean title text')
data_split

/N/u/tanakulk/Carbonate/.conda/envs/CV/lib/python3.8/site-packages/bs4/__init__.py:414: MarkupResemblesLocatorWarning: "http://127.0.0.1:8500/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


,Id,Title,Body,Tags,clean body text,clean body text tokens,clean title text,clean title text tokens
0,1,How to check if an uploaded file is an image w...,<p>I'd like to check if an uploaded file is an...,php image-processing file-upload upload mime-t...,"[like, check, uploaded, file, image, file, e, ...",like check upload file image file e g png jpg...,"[check, uploaded, file, image, without, mime, ...",check upload file image without mime type
1,2,How can I prevent firefox from closing when I ...,"<p>In my favorite editor (vim), I regularly us...",firefox,"[favorite, editor, vim, regularly, use, ctrl, ...",favorite editor vim regularly use ctrl w exec...,"[prevent, firefox, closing, press, ctrl, w]",prevent firefox close press ctrl w
2,3,R Error Invalid type (list) for variable,<p>I am import matlab file and construct a dat...,r matlab machine-learning,"[import, matlab, file, construct, data, frame,...",import matlab file construct data frame matla...,"[r, error, invalid, type, list, variable]",r error invalid type list variable
3,4,How do I replace special characters in a URL?,"<p>This is probably very simple, but I simply ...",c# url encoding,"[probably, simple, simply, find, answer, basic...",probably simple simply find answer basicaly w...,"[replace, special, characters, url]",replace special character url
4,5,How to modify whois contact details?,<pre><code>function modify(.......)\n{\n $mco...,php api file-get-contents,"[function, modify, mcontact, file, get, conten...",function modify mcontact file get content htt...,"[modify, whois, contact, details]",modify whois contact detail
...,...,...,...,...,...,...,...,...
197719,199997,Custom clipping path over HTML <div>,<p>After unsuccessful experimentation with SVG...,javascript jquery css svg mask,"[unsuccessful, experimentation, svg, looking, ...",unsuccessful experimentation svg look use ano...,"[custom, clipping, path, html]",custom clip path html
197720,199998,git alias for HEAD:refs/for/master,<p>I am configuring <code>Gerrit</code> and I ...,git gerrit,"[configuring, gerrit, would, like, avoid, writ...",configure gerrit would like avoid write git p...,"[git, alias, head, refs, master]",git alias head ref master
197721,199999,Can i prevent the keyboard to push things up w...,<p>I am using sencha touch beta 3 version at t...,android ios sencha-touch sencha sencha-touch-2,"[using, sencha, touch, beta, 3, version, momen...",use sencha touch beta 3 version moment header...,"[prevent, keyboard, push, things, shown]",prevent keyboard push thing show
197722,200000,Excel time dd/hh/mm conversion to minutes only,<p>How do I convert turn around time written i...,excel excel-formula,"[convert, turn, around, time, written, excel, ...",convert turn around time write excel cell 0d ...,"[excel, time, dd, hh, mm, conversion, minutes]",excel time dd hh mm conversion minute


Total number of tags and unique tags in 200k training data

In [37]:
def get_unique_tags(df):
  tags = df['Tags'].str.cat(sep=' ')
  tags = list(tags.split(" "))
  print("Total tags present",len(tags))
  print("Total unique tags present:",len(list(set(tags))))
  return tags

In [38]:
tags = get_unique_tags(data_split)

Total tags present 571038
Total unique tags present: 23686


In [39]:
tag_count = {}
unique_tags = list(set(tags))

for tag in unique_tags:
  #print(tag)
  tag_count[tag] = tags.count(tag)

Creating a dataframe of unique tags and the number of times each tag is appears in the answer.

In [40]:
sorted_tags = pd.DataFrame(sorted(tag_count.items(), key=lambda item: item[1],reverse=True),columns=['tags','count'])
sorted_tags

,tags,count
0,c#,15288
1,java,13495
2,php,12811
3,javascript,12039
4,android,10503
...,...,...
23681,activator,1
23682,ruby1.9,1
23683,fpic,1
23684,pointer-arithmetic,1


Dropping all tags except top 500 tags from every record

In [41]:
top500_tags = sorted_tags['tags'][:500]
type(top500_tags)
type(top500_tags)
top500_tags_list = list(top500_tags)

#top500_tags_list

In [42]:
def find_most_frequent_tags(x):
  #print(x)
  list1 = x.split(" ")
  result = ""
  for tag in list1:
    #print(tag)
    if tag in top500_tags_list:
      #print("in")
      result+= " " + tag
  return result


In [43]:
data_tags = data_split['Tags'].apply(lambda x: find_most_frequent_tags(x))
print("after taking top 500 frequency tags",data_tags.shape)
data_tags = data_tags.apply(lambda x: x if len(x)>0 else None)
data_new = pd.concat([data_split[['clean title text tokens','clean body text tokens']],data_tags],axis=1)
print(data_new.shape)
print(data_new)
data_new.reset_index(drop=True)
data_new.dropna(subset= ['Tags'],inplace = True)
print("After dropping tags with 0 length",data_new.shape)
data_new.reset_index(drop= True)
print("data new is", data_new)
type(data_new.head())

after taking top 500 frequency tags (197724,)
(197724, 3)
                           clean title text tokens  \
0        check upload file image without mime type   
1               prevent firefox close press ctrl w   
2               r error invalid type list variable   
3                    replace special character url   
4                      modify whois contact detail   
...                                            ...   
197719                       custom clip path html   
197720                   git alias head ref master   
197721            prevent keyboard push thing show   
197722       excel time dd hh mm conversion minute   
197723                make sunspot solr start work   

                                   clean body text tokens  \
0        like check upload file image file e g png jpg...   
1        favorite editor vim regularly use ctrl w exec...   
2        import matlab file construct data frame matla...   
3        probably simple simply find answer basic

pandas.core.frame.DataFrame

Vectorizing labels (tags) with 0 or 1 (binary vectorizer)

In [73]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vec = CountVectorizer(tokenizer = lambda x: x.split(), binary='true')
y = vec.fit_transform(data_new['Tags'])

Feature extracting using tfidf for both body and titles. We will get sparse matrics as out after fit and transform

In [74]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_body = TfidfVectorizer(min_df=0.00009, max_features=20000, smooth_idf=True, norm="l2",tokenizer = lambda x: x.split(), sublinear_tf=False,ngram_range=(1,4))
tfidf_title = TfidfVectorizer(min_df=0.00009, max_features=20000, smooth_idf=True, norm="l2",tokenizer = lambda x: x.split(), sublinear_tf=False,ngram_range=(1,4))
x_train_tfidf_body = tfidf_body.fit_transform(data_new['clean body text tokens'])
x_train_tfidf_title = tfidf_title.fit_transform(data_new['clean title text tokens'])

Stacking body and title together

In [75]:
from scipy.sparse import hstack
x_data = hstack([x_train_tfidf_body,x_train_tfidf_title])

Splitting data into train and test 

In [76]:
from sklearn.model_selection import train_test_split
x_train, x_test,y_train, y_test = train_test_split(x_data, y, test_size=0.2,random_state = 0)

In [77]:
print("shape of train data X:",x_train.shape, "Y :",y_train.shape)
print("shape of test data X:",x_test.shape,"Y:",y_test.shape)

shape of train data X: (141809, 28915) Y : (141809, 500)
shape of test data X: (35453, 28915) Y: (35453, 500)


Training and testing a Binary Relevance classifier using Random Forest

In [51]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.ensemble import RandomForestClassifier
import time

startbr=time.time()

br = BinaryRelevance(classifier = RandomForestClassifier(), require_dense = [False, True])

br.fit(x_train, y_train)

print("Time taken to train :", round(time.time()-startbr,0))

Time taken to train : 31725.0


In [52]:
br_pred = br.predict(x_test)

print("Accuracy: ", accuracy_score(y_test, br_pred))
print("Hamming loss: ", hamming_loss(y_test, br_pred))

Accuracy:  0.1568837615998646
Hamming loss:  0.003102812173863989


In [53]:
from sklearn.metrics import classification_report

precision = precision_score(y_test, br_pred, average='micro')
recall = recall_score(y_test, br_pred, average='micro')
f1 = f1_score(y_test, br_pred, average='micro')
 
print("Micro-average metrics: ")
print("Precision: ", precision, "Recall: ", recall, " F1-measure: ", f1)

precision = precision_score(y_test, br_pred, average='macro')
recall = recall_score(y_test, br_pred, average='macro')
f1 = f1_score(y_test, br_pred, average='macro')
 
print("Macro-average quality numbers: ")
print("Precision: ", precision, "Recall: ", recall, " F1-measure: ", f1)

print (classification_report(y_test, br_pred))


Micro-average metrics: 
Precision:  0.853010411951109 Recall:  0.26690557806170145  F1-measure:  0.40658984981874674
Macro-average quality numbers: 
Precision:  0.3687612721296166 Recall:  0.09539278928057135  F1-measure:  0.1318538570293653
              precision    recall  f1-score   support

           0       0.75      0.36      0.48       152
           1       0.52      0.01      0.02      1081
           2       0.00      0.00      0.00        31
           3       0.00      0.00      0.00        47
           4       0.00      0.00      0.00        39
           5       0.00      0.00      0.00        46
           6       0.00      0.00      0.00        82
           7       0.00      0.00      0.00        46
           8       0.93      0.29      0.45       170
           9       0.85      0.18      0.29        62
          10       0.00      0.00      0.00        64
          11       0.00      0.00      0.00        57
          12       1.00      0.03      0.05        39
 

/N/u/tanakulk/Carbonate/.conda/envs/CV/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/N/u/tanakulk/Carbonate/.conda/envs/CV/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/N/u/tanakulk/Carbonate/.conda/envs/CV/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resul

Predicting a sample using the built model

In [78]:
test1 = ['problem eclipse regard xml file eclipse complain android scrollbars android fadingedge allow string check android developer site fact accept string xml file relate question pose problem miss android xmlns see code line begin xmlns correct believe complete file content xml version 1 0 encode utf 8 com example todolist todolistitemview xmlns android http schema android com apk res android android layout width fill parent android layout height fill parent android pad 10dp android scrollbars verticle android textcolor color notepad text android fadingedge verticle']
tfidf_test = TfidfVectorizer(min_df=0.00009, max_features=20000, smooth_idf=True, norm="l2",tokenizer = lambda x: x.split(), sublinear_tf=False,ngram_range=(1,4))
test2 = ['eclipse complain android scrollbars android fadingedge allow string include code']

qs = tfidf_body.transform(test1)
# print(qs.shape)
qs2 = tfidf_title.transform(test2)
# print(qs2.shape)
q = hstack([qs,qs2])

Predicted Tags:

In [79]:
op = br.predict(q)
print(op)

  (0, 19)	1
  (0, 133)	1
  (0, 495)	1


In [80]:
print(vec.get_feature_names()[19])
print(vec.get_feature_names()[133])
print(vec.get_feature_names()[495])

android
eclipse
xml
